# Busca em grafo - algoritmo de Dijstra


#### Referências

- https://theory.stanford.edu/~amitp/GameProgramming/AStarComparison.html


#### Código


In [1]:
import numpy as np
from itertools import product


In [25]:
class A_start:
    def __init__(self, dimension: tuple[int, int]) -> None:
        self.dimension = dimension
        self.x_values: list[int] = list(range(1, self.dimension[0] + 1))
        self.y_values: list[int] = list(range(1, self.dimension[1] + 1))
        self.xy_pairs: list[tuple[int, int]] = list(
            product(self.x_values, self.y_values)
        )
        self._iterations: list[tuple[int, int]] = [(1, 0), (-1, 0), (0, 1), (0, -1)]

    def reset_nodes(self) -> None:
        self.nodes: list[dict] = []
        for x, y in self.xy_pairs:
            self.nodes.append(self.gen_nodes(x, y))
        self.nodes[self.start_id]["distance"] = 0

    def reset_sets(self) -> None:
        self.close: set[int] = set()
        self.open: set[int] = set()
        self.open.add(self.start_id)

    def set_problem(self, start: tuple[int, int], target: tuple[int, int]) -> None:
        self.start = start
        self.target = target
        self.start_id = self.xy_pairs.index(self.start)
        self.target_id = self.xy_pairs.index(self.target)

        self.reset_nodes()
        self.reset_sets()

    def gen_nodes(self, x: int, y: int) -> dict:
        node_id = self.xy_pairs.index((x, y))
        node = {
            "id": node_id,
            "x": x,
            "y": y,
            "neighbors": self.gen_neighbors(node_id, x, y),
            "distance": np.inf,
            "predecessor": None,
        }
        return node

    def inside_grid(self, index_x: int, index_y: int) -> bool:
        return 0 <= index_x < self.dimension[0] and 0 <= index_y < self.dimension[1]

    def distance_grid(self, node_id: int, neighbor_id: int) -> float:
        x, y = self.xy_pairs[node_id]
        xn, yn = self.xy_pairs[neighbor_id]

        if (y, yn) in [(5, 6), (6, 5)] and (x, xn) in [
            (3, 3),
            (4, 4),
            (5, 5),
            (6, 6),
            (7, 7),
        ]:
            return np.inf

        if (x, xn) in [(5, 6), (6, 5), (7, 8), (8, 7)]:
            return 5

        if (y, yn) in [(5, 6), (6, 5)]:
            return 15

        return 10

    def gen_neighbors(self, node_id: int, x: int, y: int) -> dict:
        index_x = self.x_values.index(x)
        index_y = self.y_values.index(y)

        neighbors = dict()
        for xi, yi in self._iterations:
            new_index_x = index_x + xi
            new_index_y = index_y + yi

            if not self.inside_grid(new_index_x, new_index_y):
                continue

            neighbor = (
                self.x_values[new_index_x],
                self.y_values[new_index_y],
            )
            neighbor_id = self.xy_pairs.index(neighbor)
            if (distancia := self.distance_grid(node_id, neighbor_id)) == np.inf:
                continue
            neighbors[neighbor_id] = distancia
        return neighbors

    def get_neighbors(self, id: int) -> list[int]:
        yield from [n for n in self.nodes[id]["neighbors"]]

    def get_route(self) -> list[tuple[int, int]]:
        route = []
        current = self.target_id
        route.append(self.xy_pairs[current])

        while current != self.start_id:
            current = self.nodes[current]["predecessor"]
            route.append(self.xy_pairs[current])
        route.reverse()

        return route

    def h_cost(self, id: int) -> float:
        x, y = self.xy_pairs[id]
        return abs(x - self.target[0]) + abs(y - self.target[1])

    def g_cost(self, id: int) -> float:
        return self.nodes[id]["distance"]

    def movement_cost(self, id: int, n_id: int) -> float:
        return self.nodes[id]["neighbors"][n_id]

    def best_n_from_open(self) -> int:
        return min(self.open, key=lambda id: self.nodes[id]["distance"])

    def execute(self):
        current = self.start_id
        while current != self.target_id:
            current = self.best_n_from_open()
            self.open.discard(current)
            self.close.add(current)

            for n in self.get_neighbors(current):
                cost = self.g_cost(current) + self.movement_cost(current, n)

                if self.nodes[n]["distance"] > cost or (
                    (n not in self.close) and (n not in self.open)
                ):
                    self.close.discard(n)
                    self.nodes[n]["distance"] = cost + self.h_cost(n)
                    self.nodes[n]["predecessor"] = current
                    self.open.add(n)


In [26]:
problem = A_start((10, 10))

In [27]:
problem.set_problem((9, 2), (1, 9))

In [28]:
problem.execute()

In [29]:
problem.get_route()

[(9, 2),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 6),
 (8, 6),
 (7, 6),
 (6, 6),
 (6, 7),
 (6, 8),
 (5, 8),
 (4, 8),
 (3, 8),
 (2, 8),
 (1, 8),
 (1, 9)]